# **MDX 데이터 분석하기**
https://pypi.org/project/mdict-utils/

## **1 레시피 SQlite 데이터 불러오기**
작업한 내용을 대상으로 중분류 내용 추가하기
- 법률, 식품, 자동차 등 중분류 내용이 당장은 필요가 없어 보인다.
- 작업을 진행하면서 필요하면 추가하기

In [1]:
# 레시피 데이터베이스 불러오기
from muyong.nlp import mdx_to_df
df    = mdx_to_df('backup/recipe.db')
menus = df.entry.values.tolist()
len(menus), " ".join(menus)[:200]

(908,
 '가오리조림 가오리찜 가자미야채탕 가자미찜 가지냉채 가지불고기 가지선 가지오징어냉채 가지조림 가지회 각색두부전골 각색버섯덮밥 간장 갈비구이 갈비구이찜 갈비찜 갈비탕 감자가자미구이 감자갈비탕 감자고로케 감자그라땡 감자대구오븐구이 감자두부조림 감자막튀김 감자맑은국 감자밥 감자베이컨볶음 감자보트샐러드 감자볶음 감자볶음밥 감자볶음밥치즈구이 감자부침 감자새우그라땡 감')

In [2]:
food_recipes = df.paraphrase.values.tolist()
result = []
import re
for _ in food_recipes:
    result += re.findall("[가-힣]+", _)
len(result), len(list(set(result)))

(88636, 10574)

In [3]:
from konlpy.tag import Hannanum, Okt, Mecab, Komoran
food_text = " ".join(list(set(result)))
food_tokens = Mecab().nouns(food_text)
len(food_tokens)

9514

In [4]:
from nltk import Text
food_tokens_text = Text(food_tokens)
food_nouns = [_[0]  for _ in food_tokens_text.vocab().most_common()]
len(food_nouns)

2626

## **2 식재료 관련 데이터 찾기**
[전은경  맛을 표현하는 단어 (2003)](https://m.blog.naver.com/PostView.nhn?blogId=seongho0805&logNo=150048557298&proxyReferer=https%3A%2F%2Fwww.google.com%2F)

In [5]:
import pickle
with open('data/nerDict.pk', 'rb') as handle:
    nerDict = pickle.load(handle)

In [6]:
query     = "한천"  # 원문 내용의 확인
q_data    = nerDict[nerDict.Text == query].Data.values.tolist()
q_data

[[('명사', ['1', '몹시 가문 여름 하늘.']), ('명사', ['2', '몹시 가문 날씨.\r\n'])],
 [('명사',
   ['1', '겨울의 차가운 하늘.', '¶ 뿌옇게 흐렸던 한천에서 희끗희끗 눈발이 날리기 시작했다.≪이원규, 훈장과 굴레≫']),
  ('명사', ['2', ' =한절02.\r\n'])],
 [('명사', ['명사', '찬물이 솟는 샘.\r\n'])],
 [('명사', ['명사', '땀을 흘리며 헐떡거림.', '한천-하다01(汗喘--) [한ː---]']),
  ('동사', ['동사', '땀을 흘리며 헐떡거리다.\r\n'])],
 [('명사', ['명사', ' =우무01. ‘우무01’, ‘우뭇가사리’로 순화.\r\n'])],
 [('명사', ['명사', '『인명』', '‘이재04’의 호.\r\n'])]]

# **메뉴 데이터 N-Gram 분석**

## **1 N-Gram 데이터**
**itemIndexTemp** 를 활용하며 고유단어 찾기

In [1]:
# 명사 추출사전 데이터 호출
with open("data/nouns_tokens.txt", "r") as f:
    valid_token = f.read()
valid_token = valid_token.split(",")
len(valid_token)

689

In [2]:
# 분석할 메뉴데이터 호출
import pandas as pd  # 분석을 의뢰할 메뉴 데이터 [list]
menus_muyong  = pd.read_csv('data/menu_muyong.csv', encoding='ms949')
menus_muyong  = menus_muyong.dropna(subset=['메인1', '메인2'])
menus_muyong  = menus_muyong.fillna('')  # NaN 값을 지운다
tokens = []                # DataFrame 에서 Token List 추출
for _ in menus_muyong.columns[4:]:
    tokens += menus_muyong[_].values.tolist()

import re
from nltk import Text
menus_muyong = re.findall(r"[가-힣]+", " ".join(tokens))
menus_muyong = list(set(menus_muyong))
len(menus_muyong), " ".join(menus_muyong[:5])

(2880, '청경채겉절이 봉추찜닭 버섯파프리카잡채 매콤돈사태찜 날치알참치야채비빔밥')

In [3]:
# 메뉴 명사추출기
menu_token  = menus_muyong[1828]
print("원본내용:", menu_token)
from muyong.nlp import food_nouns
food_nouns(menu_token, valid_token)

원본내용: 수제하트맛살계란전


['맛살', '계란', '전']

## **2 만개의 레시피 활용**
긴 제목에서 레시피만 추출/ 나머지 구분하기

In [10]:
import pandas as pd
menus_man = pd.read_csv("data/menu_1000recipe.csv", sep="|")
menus_man = menus_man.fillna('')
menus_man = menus_man.Menu.values.tolist()
menus_man[:3]

['호불호 없는 토마토파스타는 역시 ~! 토마토냉파스타 ★',
 '달콤 촉촉 달걀 푸딩 만들기, 만드는 법 ( 부드러운 달걀 요리 기본 원리 )',
 '손이가요 손이가~~자꾸 손이 가는 [가지전]']

In [11]:
# 메뉴 명사추출기
menu_token  = menus_man[4528]
print("원본내용:", menu_token)
from muyong.nlp import food_nouns
food_nouns(menu_token, valid_token)

원본내용: 아이볶음밥 김치볶음밥 카레가루를 넣어봐요!


['볶음밥', '김치', '카레']

<br></br>
# **Naver API 로 Unique Token 묶음 분석**
메뉴의 이름들 중 대표성 높은 메뉴들 찾기

## **1 레시피 데이터 특징찾기**
잡일보단 **머신러닝 분석** 으로 우선 **모델** 만들기 (30일 발표용)
1. 구내식당은 대략 **2,300 개로** 추출 된다
1. 만개의 레시피는 50,000개로 특징이 추출
1. 우선은 **2,300 개** 로 **모델 사이트를 우선 만들어서 9월초 시연모델** 만들기

In [5]:
%%time
# # 10,000 개 레시피 분석
# result = ["_".join(sorted(food_nouns(menu, valid_token)))  
#           for menu in menus_man  
#           if "_".join(sorted(food_nouns(menu, valid_token)))]

# result = list(sorted(set(result)))
# print(len(menus_man), len(result))
# with open("backup/menus_unique_tokens_10000.txt", 'w') as f:
#     f.write(",".join(result))
# print("file saved")

# Muyong 레시피 분석
result = ["_".join(sorted(food_nouns(menu, valid_token)))  
          for menu in menus_muyong  
          if "_".join(sorted(food_nouns(menu, valid_token)))]

result = list(sorted(set(result)))
print(len(menus_muyong), len(result))
with open("backup/menus_unique_tokens_muyong.txt", 'w') as f:
    f.write(",".join(result))
print("file saved")

2880 2336
file saved
CPU times: user 658 ms, sys: 4.43 ms, total: 663 ms
Wall time: 662 ms


## **2 Sqlite3 Open API 수집 결과 확인하기**
특징을 대상으로 네이버 블로그 데이터 대상 수집하기
1. 저장결과를 Sqlite3 로 저장하기
1. 안정적인 작업을 수행하도록 잘 구성하기

In [1]:
%%time
import os, sqlite3, json, time
con = sqlite3.connect("backup/food_api_muyong.db")

import pandas as pd
df         = pd.read_sql("select * from 'foods'", con, index_col=None).drop('index', axis=1)
con.close()
food_links = df.data.values.tolist()
df.data    = [json.loads(_) for _ in food_links] # json 디코딩
index_err  = [no  for no,_ in enumerate(df.data.values.tolist())  if len(_) == 0]
print(df.iloc[index_err, :])

                title data
6          가쓰오국_어묵_유부   []
7             가쓰오국_유부   []
8          가쓰오국_유부_팽이   []
136   감자_미트볼_알감_자미_조림   []
1209       돈육_불고기_육간장   []
1362         떡_물부추_해물   []
1587       무침_물부추_콩나물   []
1666         물부추_전_해물   []
2209          오징어탕_탕수   []
CPU times: user 14.6 s, sys: 1.39 s, total: 16 s
Wall time: 15.8 s


In [16]:
%%time
import os, sqlite3, json, time
con = sqlite3.connect("backup/food_api_10000_01.db")
tables = [_[0] for _ in con.execute("SELECT name FROM sqlite_master WHERE type='table';")]
print(tables)  # 테이블 이름의 확인

import pandas as pd
df  = pd.read_sql("SELECT * FROM '{}'".format(tables[0]), con, index_col=None).drop('index', axis=1)
con.close()

# 내부 Json String 을 Decoding
food_links = df.data.values.tolist()
df.data    = [json.loads(_) for _ in food_links] # json 디코딩
index_err  = [no  for no,_ in enumerate(df.data.values.tolist())  if len(_) == 0]
print(df.iloc[index_err, :]) # 자료수집시 오류가 난 Key Word 선별

['foods']
           title data
20910  곰_배추찜_양배추   []
CPU times: user 33.3 s, sys: 2.86 s, total: 36.2 s
Wall time: 36.2 s


In [26]:
# 수집값이 없는 인덱스를 새로 수정하기 
# Token Index 값이 유용한지를 Naver OpenAPI 결과값으로 구분하는 방법도 유용할 듯!!
index_err = [no  for no,_ in enumerate(df.data.values.tolist())  if len(_) == 0]
token_temp_list = ["가쓰오_어묵_유부","가쓰오_유부","가쓰오_유부_팽이","감자_미트볼_알감자_조림",\
              "돈육_불고기_간장","떡_부추_해물","무침_부추_콩나물","부추_전_해물","오징어_탕수"]

for no, i in enumerate(index_err):
    df.iloc[i, [0]] = token_temp_list[no]

In [29]:
%%time
from muyong.naver import get_blog_list
from tqdm import tqdm
result = {}
for _ in tqdm(token_temp_list):
    temp = []
    query_token  = _+"_만들기"
    for i in range(1, 500, 100):
        temp += get_blog_list(query_token, i)['items']
    time.sleep(.3)
    result[_] = temp

100%|██████████| 9/9 [00:17<00:00,  2.00s/it]

CPU times: user 661 ms, sys: 98 ms, total: 759 ms
Wall time: 17.7 s


In [55]:
for no, i in enumerate(index_err):
    df.iloc[i, 1] = result[token_temp_list[no]]
    
import pickle
with open('data/naver_key_list.pk', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("save is done")

save is done


In [10]:
with open("menus_unique_token.txt", "r") as f:
    word_tokens = f.read().split(",")

# get_blog_list("가슴살_닭_매콤_볶음_우동"+"_만들기", 101)["items"][:2]
# len(word_tokens), word_tokens[1000]
list(result.keys())

['가쓰오_어묵_유부',
 '가쓰오_유부',
 '가쓰오_유부_팽이',
 '감자_미트볼_알감자_조림',
 '돈육_불고기_간장',
 '떡_부추_해물',
 '무침_부추_콩나물',
 '부추_전_해물',
 '오징어_탕수']

In [11]:
import pandas as pd
len(result[list(result.keys())[0]])

In [2]:
%%time
import pandas as pd
from tqdm import tqdm
from muyong.naver import get_blog_list
import time, json, os, sqlite3
file_db = "foods.db"

token_temp_list = ["가쓰오_어묵_유부","가쓰오_유부","가쓰오_유부_팽이","감자_미트볼_알감자_조림",\
              "돈육_불고기_간장","떡_부추_해물","무침_부추_콩나물","부추_전_해물","오징어_탕수"]

for _ in tqdm(token_temp_list):
    # web link 목록 수집하기
    result, temp = [], []
    query_token  = _+"_만들기"
    for i in range(1, 500, 100):
        temp += get_blog_list(query_token, i)['items']
    time.sleep(.3)
    result.append([_, temp])
    
    # sqlite3 저장하기 (단어별 결과를 Json으로 저장 (list는 저장시 오류))
    df = pd.DataFrame(result)
    df.columns = ['title', 'data']
    df.data    = [ json.dumps(_)  for _ in df.data ]
    con = sqlite3.connect(file_db)
    df.to_sql('foods', con, if_exists='append')
    con.commit()
    time.sleep(.1)
    
df

,title,data
0,오징어_탕수,"[{""title"": ""[\ub3c4\uc2dd\ub2f9]<b>\ud0d5\uc21..."


## **4 링크 url 데이터만 저장하기**
**API 수집결과 중 Link Url** 만 저장하기

In [2]:
import pickle
with open('backup/temp/naver_api_org.pk', 'rb') as handle:
    df = pickle.load(handle)

# data 에서 naver 링크만 찾아서 정리하기
df.head(3)

,title,data
0,가슴살_계란_닭_장조림,[{'title': '부드러운 닭<b>가슴살</b> <b>장조림</b> <b>만들기...
1,가슴살_굴소스_닭_볶음밥,[{'title': '수비드 닭<b>가슴살</b> 요리 : 돈까스 &amp; <b>...
2,가슴살_닭_라이스_카레,[{'title': '닭의품격 초간편 냉동닭<b>가슴살</b> <b>카레</b><b...


In [3]:
result = []
from tqdm import tqdm
for datum in tqdm(df.data.values.tolist()):
    temp = []
    for _ in datum:  # 500개 개별확인
        if _['link'].find("https://blog.naver.com/") != -1:
            temp.append(_['link'])
        elif len(_) < 3:
            print("데이터가 없습니다"); pass
    result.append("|".join(temp))
df.data = result
df.to_csv('data/naver_links_muyong.csv', index=None)
df.head()

100%|██████████| 2335/2335 [00:00<00:00, 4734.80it/s]


,title,data
0,가슴살_계란_닭_장조림,https://blog.naver.com/berry__chu?Redirect=Log...
1,가슴살_굴소스_닭_볶음밥,https://blog.naver.com/happyday82?Redirect=Log...
2,가슴살_닭_라이스_카레,https://blog.naver.com/hogengggy?Redirect=Log&...
3,가슴살_닭_매콤_볶음_우동,https://blog.naver.com/cutylee82?Redirect=Log&...
4,가슴살_닭_볶음밥_파인애플,https://blog.naver.com/aurelly3280?Redirect=Lo...


In [17]:
err_index = [no for no, _ in enumerate(df.data)   if len(_.split('|')) < 10]
err_index

[369, 453, 880, 914]

In [30]:
df_sum = df.drop(err_index, axis=0).reset_index(drop=True)
df_sum.to_csv("data/naver_links_muyong.csv", index=None)
df.shape, df_sum.shape

((2335, 2), (2331, 2))

In [21]:
from tqdm import tqdm
from muyong.naver import get_blog_list
def error_api(token_list):
    for _ in tqdm(token_list):
        # web link 목록 수집하기
        result, temp = [], []
        query_token  = _+"_만들기"
        for i in range(1, 500, 100):
            temp += get_blog_list(query_token, i)['items']
        time.sleep(.3)
        result.append([_, temp])
    return result

In [ ]:
token_temp_list = ["계란찜_알새우", "고추_마늘_무침", "깐풍_어묵", "깻잎_볶음_나물"]
error_data = error_api(token_temp_list)

In [5]:
len(df.data[0].split('|'))

487

In [ ]:
# Today
# 단어 Token 을 나열된 순서대로 정리하여 문법구조 분석 알고리즘 적용
# CFG 와 유사한 방식으로 불필요한 단어 및 묶음 정리하기
# 묶음 기준을 설정한 뒤, 크롤링을 통해 단어별 문서 300개를 수집 분석하여 연관단어 찾기

명엽채(명태채)/청포북(녹두묵)

1. 석박지/무석박지

```
마요 => 마요네즈
소시지 => 소세지
꺳잎 => 깻잎
게란 => 계란
배춧 => 배추
불낙 => 낙지
콘 => 옥수수
돈 => 돼지
탕수 => 탕수육
우무 => 우묵
중국식/
한국식/
```

<br></br>
# **Naver Post Detail Crawling**
메뉴의 이름들 중 대표성 높은 메뉴들 찾기

## **1 수집결과 살펴보기**
Sqlite 결과를 보다 활용하기 쉽게 저장하기

In [1]:
# import pandas as pd
# links_org      = pd.read_csv('backup/naver_link_temp.csv')
# links_org      = links_org.fillna('')

# import sqlite3
# conn = sqlite3.connect("backup/naver.db")
# df = pd.read_sql('SELECT * FROM "foods"', conn, index_col=None).drop('index', axis=1)
# conn.close()
# df.columns = links_org.columns
# df.title   = links_org.title[:df.shape[0]].tolist()
# df.to_csv("data/naverPost2.csv", index=None)
# df.head(2)

In [14]:
import pandas as pd
from glob import glob
file_list = glob("data/naverPost*.*")
result    = []
for _ in file_list:
    result.extend(pd.read_csv(_).values.tolist())

df = pd.DataFrame(result)
df.columns = ['Query', 'Posts']
df.to_csv("data/naverPost.csv", index=None)
df.head(2)

,Query,Posts
0,가슴살_계란_닭_장조림,반찬 볶음 조림.부드러운 닭가슴살 장조림 만들기.베리츄..2017.9.1.22 32...
1,가슴살_굴소스_닭_볶음밥,아임닭 7기 종료.수비드 닭가슴살 요리 돈까스 볶음밥 만들기.주니호맘..2019.4...


# **Naver Post Detail Crawling**
메뉴의 이름들 중 대표성 높은 메뉴들 찾기

## **1 수집결과 살펴보기**
Sqlite 결과를 보다 활용하기 쉽게 저장하기

In [11]:
pd.read_csv(file_list[0]).head(2)

,title,data
0,가슴살_계란_닭_장조림,반찬 볶음 조림.부드러운 닭가슴살 장조림 만들기.베리츄..2017.9.1.22 32...
1,가슴살_굴소스_닭_볶음밥,아임닭 7기 종료.수비드 닭가슴살 요리 돈까스 볶음밥 만들기.주니호맘..2019.4...


In [3]:
df.tail()

,title,data
824,돈육_두부구이_양념_조림,두부양념구이 필수 밑반찬 두부양념조림 만들기.매일매일 가족밥상..홈베이킹 and 요...
825,돈육_떡_엿_장조림,생활정보.김수미 스테이크밥버거 레시피 수미네반찬 돈육표고장조림 만드는법 김수미 김소...
826,돈육_떡찜,[10월 15일] 시골된장찌개 매운돈사태떡찜 연근커틀릿 땅콩버터진미채 국내산오징어젓...
827,돈육_매운_찜,쿰척쿰척 먹방.팜앤포크 삼겹살 목살 택배 무항생제 돼지고기 쪽갈비김치찜 만드는법.먹...
828,돈육_매콤_사태_찜,내가 만든 점심메뉴.떡넣은 돼지고기사태찜 만들기.오봉..2017.12.26.19 5...


In [16]:
link_temp = links_org.iloc[370:1200,:] # .to_csv('naver_links_temp.csv')
link_temp = link_temp.reset_index(drop=True)
link_temp = link_temp.drop([544], axis=0).reset_index(drop=True)
link_temp.head(2)

,title,data
0,계란찜_야채,https://blog.naver.com/osn1201?Redirect=Log&lo...
1,계란찜_야채_옥수수,https://blog.naver.com/kkalgid1?Redirect=Log&l...
